In [1]:
import Pkg; 

cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol

using PyCall
using JLD2
using LinearAlgebra
using Rotations
using GLMakie

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolHelpers`
[ Info: Precompiling MorphoMol [85e20460-a9b2-48f6-9df6-e154e9748d83]
[ Info: Precompiling GLMakie [e9467ef8-e4e7-5192-8a1a-b1aee30e663a]
Precompiling DistributionsTestExt
  ✓ Distributions → DistributionsTestExt
  1 dependency successfully precompiled in 5 seconds. 43 already precompiled.
[ Info: Precompiling DistributionsTestExt [ffbe0ea5-a612-5ff7-aaf5-cac02eef3019]
┌ Warning: Module Distributions with build ID fafbfcfd-03ee-2362-0000-a5b5316b58ab is missing from the cache.
│ This may mean Distributions [31c24e10-a181-5473-b8eb-7969acd0382f] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948
[ Info: Skipping precompilation since __precompile__(false). Importing DistributionsTestExt [ffbe0ea5-a612-5ff7-aaf5-cac02eef3019].


In [19]:
py"""
import oineus as oin
import numpy as np
import torch
import diode

def get_diagram(points, dim):
    points = np.asarray(points)
    simplices = diode.fill_alpha_shapes(points)
    fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])
    # no cohomology
    dualize = False
    # create VRU decomposition object, does not perform reduction yet
    dcmp = oin.Decomposition(fil, dualize)
    rp = oin.ReductionParams()
    rp.compute_u = rp.compute_v = True
    rp.n_threads = 1
    # perform reduction
    dcmp.reduce(rp)
    dgm_dim = dcmp.diagram(fil, include_inf_points=False)[dim]
    return dgm_dim
"""

function get_diagram(points::Vector{Vector{Float64}}, dim::Int = 1)
    py"get_diagram"(points, dim)
end

function get_total_persistence(dgm::Matrix{Float64})
    sum(dgm[:,2] - dgm[:,1])
end


get_total_persistence (generic function with 1 method)

In [12]:
function txt_to_points(input_file)
    [[parse(Float64, s) for s in split(p," ")] for p in readlines(input_file)[3:end]]
end

txt_to_points (generic function with 1 method)

In [13]:
function get_diagrams(folder)
    files = readdir(folder)
    n = length(files)
    [get_diagram(txt_to_points("$(folder)$(i-1).txt")) for i in 1:n]
end

function get_tps(dgms)
    [get_total_persistence(dgm) for dgm in dgms]
end

function get_fsols(folder)
    files = readdir(folder)
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[6]) for i in 1:n]
end

function get_thetas(folder)
    files = readdir(folder)
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[7]) for i in 1:n]
end

get_thetas (generic function with 1 method)

In [5]:
function folder_of_txts_to_jld2_with_persistence_computation(folder)
    mol_type = split(folder, "/")[end-2]
    idx = split(folder, "/")[end-1]
    dgms = get_diagrams(folder)
    tps = get_tps(dgms)
    fsols = get_fsols(folder)
    thetas = get_thetas(folder)
    @save "../../Data/measure_paths/jld2s/$(mol_type)/$(idx).jld2" dgms tps fsols thetas
end

folder_of_txts_to_jld2_with_persistence_computation (generic function with 1 method)

In [7]:
# folder = "../../Data/measure_paths/txts/6r7m/"
# for subdir in readdir(folder)
#     if subdir != ".DS_Store"
#         folder_of_txts_to_jld2_with_persistence_computation("$(folder)$(subdir)/")
#     end
# end

In [91]:
# folder = "../../Data/measure_paths/txts/6r7m/1/"
# dgms = get_diagrams(folder)
# xs = [i for i in 1:length(dgms)]
# tps = get_tps(dgms)
# fsols = get_fsols(folder)
# thetas = get_thetas(folder);

@load "../../Data/measure_paths/jld2s/6r7m/1.jld2" dgms tps fsols thetas
xs = [i for i in 1:length(dgms)];

In [92]:
f = Figure(fontsize = 7)
pd_ax = Axis(f[1, 1:3], title = "Persistence Diagram")
tp_ax = Axis(f[2, 1], title = "Total Persistence")
theta_ax = Axis(f[2, 2], title = L"\Theta")
fsol_ax = Axis(f[2, 3], title = L"F_{sol}")

sl_i = Slider(f[3, 1:3], range = 1:length(dgms), startvalue = 1)

x = sl_i.value

tp_mark = @lift(Point2f($x, $@lift(tps[$x])))
theta_mark = @lift(Point2f($x, $@lift(thetas[$x])))
fsol_mark = @lift(Point2f($x, $@lift(fsols[$x])))

dgm_points = @lift([Point2f(dgms[$x][i,1], dgms[$x][i,2]) for i in 1:size(dgms[$x])[1]])

ms = 5
scatter!(pd_ax, dgm_points, markersize = ms)

scatter!(tp_ax, xs, tps, color=:red, markersize = ms)
scatter!(tp_ax, tp_mark, color = :black, markersize = 10)

scatter!(theta_ax, xs, thetas, color=:magenta, markersize = ms)
scatter!(theta_ax, theta_mark, color = :black, markersize = 10)

scatter!(fsol_ax, xs, fsols, color=:orange, markersize = ms)
scatter!(fsol_ax, fsol_mark, color = :black, markersize = 10)

display(f)


GLMakie.Screen(...)

In [71]:
py"""
import oineus as oin
import numpy as np
import torch
import diode

def get_simplices(points):
    points = np.asarray(points)
    simplices = diode.fill_alpha_shapes(points)
    return simplices
"""
function get_simplices(points)
    py"get_simplices"(points)
end

function get_interface(simplices, n)
    #WARNING ONLY WORKS FOR TWO SUBUNITS
    tetras = [s for (s,_) in simplices if length(s) == 4]
    interface_tetras = [s for s in tetras if any([e < n for e in s]) && any([e >= n for e in s])]
    [(s, d) for (s, d) in simplices if any([issubset(s,t) for t in interface_tetras])]
end

py"""
import oineus as oin
import numpy as np
import torch
import diode

def get_diagram_from_simplices(simplices, dim):
    fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])
    # no cohomology
    dualize = False
    # create VRU decomposition object, does not perform reduction yet
    dcmp = oin.Decomposition(fil, dualize)
    rp = oin.ReductionParams()
    rp.compute_u = rp.compute_v = True
    rp.n_threads = 1
    # perform reduction
    dcmp.reduce(rp)
    dgm_dim = dcmp.diagram(fil, include_inf_points=False)[dim]
    return dgm_dim
"""

function get_diagram_from_simplices(simplices::Vector{Tuple{Vector{Int64}, Float64}}, dim::Int = 1)
    py"get_diagram_from_simplices"(simplices, dim)
end

get_diagram_from_simplices (generic function with 2 methods)

In [50]:
@load "../../Data/jld2/single_subunits/6r7m_protor.jld2" template_mol template_radii x_init
n = length(template_radii)

function get_interface_diagrams(folder)
    files = readdir(folder)
    num_of_diags = length(files)
    [get_diagram_from_simplices(get_interface(get_simplices(txt_to_points("$(folder)$(i-1).txt")), n)) for i in 1:num_of_diags]
end

get_interface_diagrams (generic function with 1 method)

In [96]:
folder = "../../Data/measure_paths/txts/6r7m/1/"
#interface_dgms = get_interface_diagrams(folder)
xs = [i for i in 1:length(interface_dgms)]
tps = get_tps(interface_dgms)
fsols = get_fsols(folder)
thetas = get_thetas(folder);

In [94]:
f = Figure(fontsize = 7)
pd_ax = Axis(f[1, 1:3], title = "Persistence Diagram")
tp_ax = Axis(f[2, 1], title = "Total Persistence")
theta_ax = Axis(f[2, 2], title = L"\Theta")
fsol_ax = Axis(f[2, 3], title = L"F_{sol}")

sl_i = Slider(f[3, 1:3], range = 1:length(interface_dgms), startvalue = 1)

x = sl_i.value

tp_mark = @lift(Point2f($x, $@lift(tps[$x])))
theta_mark = @lift(Point2f($x, $@lift(thetas[$x])))
fsol_mark = @lift(Point2f($x, $@lift(fsols[$x])))

dgm_points = @lift([Point2f(interface_dgms[$x][i,1], interface_dgms[$x][i,2]) for i in 1:size(interface_dgms[$x])[1]])

ms = 5
scatter!(pd_ax, dgm_points, markersize = ms)

scatter!(tp_ax, xs, tps, color=:red, markersize = ms)
scatter!(tp_ax, tp_mark, color = :black, markersize = 10)

scatter!(theta_ax, xs, thetas, color=:magenta, markersize = ms)
scatter!(theta_ax, theta_mark, color = :black, markersize = 10)

scatter!(fsol_ax, xs, fsols, color=:orange, markersize = ms)
scatter!(fsol_ax, fsol_mark, color = :black, markersize = 10)

display(f)


GLMakie.Screen(...)

In [99]:
file = "../../Data/measure_paths/txts/6r7m/1/0.txt"
points = txt_to_points(file)
complete = get_simplices(points)
interface = get_interface(complete, n)

c_diag = get_diagram_from_simplices(complete)
sum(c_diag[:,2] - c_diag[:,1])

1410.4672202718693